#**خدوم**

In [ ]:
pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 113.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.1 MB/s eta 0:00:00


In [4]:
%%writefile app.py

import streamlit as st
from huggingface_hub import InferenceClient
import PyPDF2
from openai import OpenAI
import re
# ------------------ Page UI ------------------
st.set_page_config(page_title="Khadoom ", page_icon="🤖", layout="wide")
page = st.sidebar.selectbox("Navigation", ["📄 Upload PDF", "💬 Chatbot"])
# --------------------------------------------



# ------------------ Hugging Face API Connection ------------------
client = OpenAI(
    base_url="https://huggingface.co/api/inference-proxy/together",
    api_key="hf_xxx"
)



# ------------------ PDF Upload Page ------------------
if page == "📄 Upload PDF":
    st.title("Khadoom")
    st.image("/content/khadoom logo.PNG", width=100)
    st.title("📄 Upload a PDF (FAQs, Policies, etc.)")

    uploaded_file = st.file_uploader("Upload a PDF document", type="pdf")

    if "document_text" not in st.session_state:
        st.session_state.document_text = ""

    if uploaded_file:
        pdf_reader = PyPDF2.PdfReader(uploaded_file)
        extracted_text = "\n".join([page.extract_text() for page in pdf_reader.pages if page.extract_text()])
        st.session_state.document_text = extracted_text  # Store extracted text globally
        st.success("✅ PDF uploaded and processed!")

        st.info(" khadoom is ready to  answer to customers 🤖")




# ------------------ Chatbot Page ------------------
elif page == "💬 Chatbot":
    st.title("💬 Khadoom Chatbot 🤖")
    st.image("/content/khadoom logo.PNG", width=100)

    # Ensure PDF is uploaded before chat
    if "document_text" not in st.session_state or not st.session_state.document_text:
        st.warning("⚠️ Please upload a PDF first on the 'Upload PDF' page!")
        st.stop()  # Prevents further execution




    # ------------------ System Prompt  ------------------
SYSTEM_PROMPT = """You are an AI customer support assistant named "Khadoom".
You MUST answer concisely based ONLY on the provided document.
The question of the user might not be direct, so you need to understand the user request to be able to fetch the correct answer from the pdf

### Rules:
- **Only answer from the document**.
- **DO NOT speculate or make up answers**.
- **If the answer is unknown, respond:** "I don't have that information in the document."
- **DO NOT repeat or rephrase the user’s question**.
- **DO NOT start responses with 'User:' or repeat conversation history**.
- **DO NOT include "<think>" statements or reasoning processes**.
- **DO NOT ignore polite feedback like 'That was helpful' or 'Thank you'—instead, respond with 'You're welcome.'**
- **Give direct answers in a professional tone**.
- ** You should greet the user back and say goodbye back or see you soon!"

### Examples:
**User:** Who wrote 'Hamlet'?
**Assistant:** William Shakespeare.

**User:** That was helpful.
**Assistant:** You're welcome.

**User**: This is not what I asked for.
**Assistant**: I’m sorry, let me clarify.

**User**: You're not answering my question correctly.
**Assistant**: I apologize for any confusion, I am under development, your question will be handled by a human.

**User:** What is the CEO’s name?
**Assistant:** I don’t have that information in the document.

**User:** hi
**Assistant:** hello, what can I help with?


**User:** what is your name?
**Assistant:** my name I Khadoom!

**User:** at which point does the water get frozen?
**Assistant:** :  Water freezes at 0 degrees Celsius or 32 degrees Fahrenheit.


"""

# ----------------------Chat History----------------------------
if "messages" not in st.session_state:
  st.session_state.messages = [
      {"role": "system", "content": SYSTEM_PROMPT}
  ]

for message in st.session_state.messages:
  if message["role"] == "system":
      continue
  role = "🤖" if message["role"] == "assistant" else "👤"
  st.chat_message(role).write(message["content"])

user_input = st.chat_input("Type your message...")

if user_input and st.session_state.document_text:
    st.session_state.messages.append({"role": "user", "content": user_input})

    completion = client.chat.completions.create(
        model="deepseek-ai/DeepSeek-R1",
        messages=st.session_state.messages,
        max_tokens=256
    )

    bot_reply = completion.choices[0].message.content.strip()
    bot_reply = re.sub(r"<think>.*?</think>", "", bot_reply, flags=re.DOTALL).strip()
    bot_reply = bot_reply.replace("<think>", "").replace("</think>", "").strip()  # Additional safeguard

    st.session_state.messages.append({"role": "assistant", "content": bot_reply})

    st.chat_message("👤").write(user_input)
    st.chat_message("🤖").write(bot_reply)

elif user_input:
    st.warning("⚠️ Please upload a PDF first!")


# ------------------ Sentiment Analysis Prompt ------------------
SENTIMENT_PROMPT = """Analyze the conversation and determine the customer's satisfaction level.

### **Rules:**
- **A normal Q&A conversation where the AI provides an answer → "The customer is satisfied."**
- **If the user complains, expresses frustration, or says the assistant is wrong or 'I am not happy' → "The customer is not satisfied."**
- **If the user expresses gratitude (e.g., "Thanks," "That was helpful") → "The customer is satisfied."**
- **ONLY return one of these two responses:**
  - "The customer is satisfied."
  - "The customer is not satisfied."
- **DO NOT explain your reasoning.**
- **DO NOT add any extra text or "<think>" statements.**
- **DO NOT assume dissatisfaction unless the user explicitly expresses frustration.**

### **Examples:**
**Example 1 (Satisfied)**
User: Who wrote 'Hamlet'?
Assistant: William Shakespeare.
User: That was helpful.
Assistant: You're welcome.
Output → "The customer is satisfied."

**Example 2 (Not Satisfied)**
User: You're not answering my question correctly.
Assistant: I apologize for any confusion, I am under development, your question will be handled by a human.
Output → "The customer is not satisfied."

**Example 3 (User says "I am not happy" - Not Satisfied)**
User: This is not what I asked for, I am not happy with your answer.
Assistant: I’m sorry, let me clarify.
Output → "The customer is not satisfied."

**Example 4 (Satisfied, Neutral but Helpful)**
User: Thank you, that was helpful!
Assistant: You're welcome!
Output → "The customer is satisfied."

**Example 5 (Frustrated User, Not Satisfied)**
User: This is not what I asked for.
Assistant: I’m sorry, let me clarify.
Output → "The customer is not satisfied."

**Example 6 (Short Conversation, But Answered Correctly)**
User: What is the capital of France?
Assistant: Paris.
Output → "The customer is satisfied."

Now analyze the following conversation and determine the final result:
"""


# ------------------ End Conversation Button ------------------
if st.button("End Conversation"):
  if len(st.session_state.messages) > 1:
      formatted_conversation = "\n".join([
          f"{msg['role'].capitalize()}: {msg['content']}"
          for msg in st.session_state.messages if msg["role"] != "system"
      ])

      sentiment_response = client.chat.completions.create(
          model="deepseek-ai/DeepSeek-R1",
          messages=[
                {"role": "system", "content": "You are a sentiment analysis assistant. Your task is to analyze customer satisfaction based on conversation history."+  SENTIMENT_PROMPT },
                {"role": "user", "content": formatted_conversation}
            ],
          max_tokens=50
      )

      sentiment_result = sentiment_response.choices[0].message.content.strip()
      sentiment_result = re.sub(r"<think>.*?</think>", "", sentiment_result, flags=re.DOTALL).strip()
      sentiment_result = sentiment_result.replace("<think>", "").replace("</think>", "").strip()  # Additional safeguard


      conversation_lower = formatted_conversation.lower()
      NEGATIVE_KEYWORDS = [
          "this is not what i asked for",
          "i am not happy",
          "not answering my question correctly",
          "upset",
          "angry",
          "frustrated"
      ]
      if any(keyword in conversation_lower for keyword in NEGATIVE_KEYWORDS):
          final_sentiment = "The customer is not satisfied."
      else:
          if "not satisfied" in sentiment_result:
              final_sentiment = "The customer is not satisfied."
          else:
              final_sentiment = "The customer is satisfied."

      st.subheader("Conversation Sentiment Analysis:")
      st.success(final_sentiment)

      st.session_state.messages = []
  else:
      st.warning("⚠️ No conversation available for sentiment analysis!")


Overwriting app.py


In [ ]:
pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.9 MB/s eta 0:00:00


In [ ]:
pip install ngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 82.7 MB/s eta 0:00:00


In [8]:
import ngrok
secret = "xxx"
listner = await ngrok.forward(8501, authtoken =secret)
print(listner.url())

!streamlit run app.py

https://c25e-35-240-177-29.ngrok-free.app



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.240.177.29:8501

  Stopping...
